In [1]:
from transformer_algebra import *
import torch

model, tokenizer = load_pythia_model("EleutherAI/pythia-14m")


The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [2]:
# 2. Run on next token
T = PromptedTransformer(model, tokenizer, "Once upon")
# 2. Run on next token
x = T(" a")
x

T(embed(' a'))

In [3]:

# 3. Check predictions
predict(x).summary()


"Top 5 predictions:\n  1. ' few' (2.70%, logit=1583.20)\n  2. ' new' (1.01%, logit=1582.22)\n  3. ' time' (0.97%, logit=1582.18)\n  4. ' little' (0.96%, logit=1582.16)\n  5. ' long' (0.88%, logit=1582.08)"

In [4]:
logits(x)[" time"]

<unembed(' time'), T(embed(' a'))> = 1582.18

In [5]:
expand(x)

LN^T((I + ΔB^1 + ΔB^2 T^1 + ΔB^3 T^2 + ΔB^4 T^3 + ΔB^5 T^4 + ΔB^6 T^5)embed(' a'))

In [11]:
contribs = contribution(expand(x), logits(x)[" time"])
contribs


Contributions to ⟨ time, LN(...)⟩:
  embed(' a'): -1.27 (-0.1%)
  ΔB^1: -2.52 (-0.3%)
  ΔB^2 T^1: +2.95 (+0.3%)
  ΔB^3 T^2: +19.64 (+2.1%)
  ΔB^4 T^3: +5.98 (+0.7%)
  ΔB^5 T^4: +40.43 (+4.4%)
  ΔB^6 T^5: +852.94 (+92.9%)
  Subtotal: 918.15
  Beta term: 664.03
  Total: 1582.18

In [12]:

# 5. Drill down into significant blocks
for term, value, pct in contribs.top_k(3):
    if hasattr(term, 'expand'):
        print(f"\n{term}:")
        sub = term.expand()
        print(f"  Attention + MLP: {sub}")


ΔB^6 T^5:
  Attention + MLP: ΔB^6_A T^5 + ΔB^6_M T^5

ΔB^5 T^4:
  Attention + MLP: ΔB^5_A T^4 + ΔB^5_M T^4

ΔB^3 T^2:
  Attention + MLP: ΔB^3_A T^2 + ΔB^3_M T^2


In [8]:
level1 = expand(x)
expand(level1)

LN^T((I + ΔB^1_A + ΔB^1_M + ΔB^2_A T^1 + ΔB^2_M T^1 + ΔB^3_A T^2 + ΔB^3_M T^2 + ΔB^4_A T^3 + ΔB^4_M T^3 + ΔB^5_A T^4 + ΔB^5_M T^4 + ΔB^6_A T^5 + ΔB^6_M T^5)embed(' a'))

In [13]:
level2 = expand(level1)
expand(level2)

LN^T((I + ΔB^1_A + ΔB^1_M + ΔB^2_A T^1 + ΔB^2_M T^1 + ΔB^3_A T^2 + ΔB^3_M T^2 + ΔB^4_A T^3 + ΔB^4_M T^3 + ΔB^5_A T^4 + ΔB^5_M T^4 + ΔB^6_A T^5 + ΔB^6_M T^5)embed(' a'))

In [15]:
T = PromptedTransformer(model, tokenizer, "The capital of Ireland")


x = T(" is")

# 3. Check predictions
print(predict(x).summary())

# 4. Expand and analyze
ex = expand(x)
logit = logits(x)[" Dublin"]
contrib = contribution(ex.inner, logit)
contrib


Top 5 predictions:
  1. ' the' (8.35%, logit=1583.74)
  2. ' a' (7.47%, logit=1583.63)
  3. ' one' (2.45%, logit=1582.52)
  4. ' now' (2.41%, logit=1582.50)
  5. ' not' (2.28%, logit=1582.45)


Contributions to ⟨ Dublin, LN(...)⟩:
  embed(' is'): -2.96 (-0.3%)
  ΔB^1: +5.79 (+0.6%)
  ΔB^2 T^1: +1.42 (+0.2%)
  ΔB^3 T^2: +4.81 (+0.5%)
  ΔB^4 T^3: +10.95 (+1.2%)
  ΔB^5 T^4: +24.90 (+2.7%)
  ΔB^6 T^5: +871.28 (+95.1%)
  Subtotal: 916.18
  Beta term: 661.48
  Total: 1577.66

In [22]:
contrib.terms[6]

ΔB^6 T^5

In [31]:
logit

<unembed(' Dublin'), T(embed(' is'))> = 1577.66

In [32]:
expand(logit)

<unembed(' Dublin'), T(embed(' is'))> = 1577.66